In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from numba import cuda

from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Conv1D, GlobalMaxPooling1D,LSTM

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

#dtype='float16'
#K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
#K.set_epsilon(1e-3) 

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 269564166442136308
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4557766656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11661780509653102710
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
def free_gpu_mem():
    cuda.select_device(0)
    cuda.close()

In [6]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [7]:
word_vec_size=200

In [8]:
def w2v_per_token_wiki(word_vec_size):
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))

    embeddings_index,songs=utils.get_embedding_dict(songs,'glove-wiki-gigaword-200')
    print(len(embeddings_index.keys()))
    max_words=len(embeddings_index.keys())
    X,y=np.array(songs),np.array(labels)

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)

    length = []
    for x in X:
        length.append(len(x))

    X = pad_sequences(sequences, maxlen=max(length)+2)

    embedding_matrix = np.zeros((max_words, word_vec_size))
    for word, i in tokenizer.word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
    
    
    return x_train,x_test,y_train,y_test,embedding_matrix,max(length)+2,max_words

In [9]:
x_train,x_test,y_train,y_test,embedding_matrix,max_sent_len,max_words=w2v_per_token_wiki(word_vec_size)

19704


In [10]:
def dummy(x_train,x_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(x_train, y_train)
    y_pred=dummy.predict(x_test)
    return f1_score(y_test,y_pred)

In [11]:
print(dummy(x_train,x_test,y_train,y_test))

0.43796109993293086


In [12]:
def dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [13]:
print(dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 6s - loss: 0.8112 - acc: 0.5801 - val_loss: 0.6169 - val_acc: 0.6286
Epoch 2/100
 - 3s - loss: 0.4917 - acc: 0.7529 - val_loss: 0.5766 - val_acc: 0.6442
Epoch 3/100
 - 3s - loss: 0.3248 - acc: 0.8693 - val_loss: 0.6166 - val_acc: 0.6484
Epoch 4/100
 - 3s - loss: 0.1845 - acc: 0.9453 - val_loss: 0.6425 - val_acc: 0.6535
Epoch 5/100
 - 3s - loss: 0.1217 - acc: 0.9702 - val_loss: 0.7451 - val_acc: 0.6541
Epoch 6/100
 - 3s - loss: 0.0840 - acc: 0.9811 - val_loss: 0.7570 - val_acc: 0.6547
Epoch 7/100
 - 3s - loss: 0.0730 - acc: 0.9832 - val_loss: 0.8111 - val_acc: 0.6496
Epoch 8/100
 - 3s - loss: 0.0639 - acc: 0.9823 - val_loss: 0.8933 - val_acc: 0.6538
Epoch 9/100
 - 3s - loss: 0.0620 - acc: 0.985

Epoch 86/100
 - 3s - loss: 0.0494 - acc: 0.9889 - val_loss: 2.4223 - val_acc: 0.6559
Epoch 87/100
 - 3s - loss: 0.0527 - acc: 0.9892 - val_loss: 2.3203 - val_acc: 0.6493
Epoch 88/100
 - 3s - loss: 0.0591 - acc: 0.9868 - val_loss: 2.1051 - val_acc: 0.6514
Epoch 89/100
 - 3s - loss: 0.0573 - acc: 0.9871 - val_loss: 2.3091 - val_acc: 0.6442
Epoch 90/100
 - 3s - loss: 0.0588 - acc: 0.9868 - val_loss: 2.1315 - val_acc: 0.6584
Epoch 91/100
 - 3s - loss: 0.0709 - acc: 0.9865 - val_loss: 2.4171 - val_acc: 0.6547
Epoch 92/100
 - 3s - loss: 0.0557 - acc: 0.9889 - val_loss: 2.3205 - val_acc: 0.6493
Epoch 93/100
 - 3s - loss: 0.0433 - acc: 0.9892 - val_loss: 2.3096 - val_acc: 0.6508
Epoch 94/100
 - 3s - loss: 0.0562 - acc: 0.9880 - val_loss: 2.1949 - val_acc: 0.6532
Epoch 95/100
 - 3s - loss: 0.0507 - acc: 0.9895 - val_loss: 2.4400 - val_acc: 0.6478
Epoch 96/100
 - 3s - loss: 0.0490 - acc: 0.9904 - val_loss: 2.5350 - val_acc: 0.6541
Epoch 97/100
 - 3s - loss: 0.0536 - acc: 0.9883 - val_loss: 2.337

In [14]:
def dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [15]:
print(dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 6s - loss: 0.8831 - acc: 0.5618 - val_loss: 0.6377 - val_acc: 0.6271
Epoch 2/100
 - 3s - loss: 0.5176 - acc: 0.7268 - val_loss: 0.5772 - val_acc: 0.6514
Epoch 3/100
 - 3s - loss: 0.3320 - acc: 0.8720 - val_loss: 0.5903 - val_acc: 0.6614
Epoch 4/100
 - 3s - loss: 0.2069 - acc: 0.9285 - val_loss: 0.6270 - val_acc: 0.6590
Epoch 5/100
 - 3s - loss: 0.1277 - acc: 0.9666 - val_loss: 0.7150 - val_acc: 0.6529
Epoch 6/100
 - 3s - loss: 0.0824 - acc: 0.9790 - val_loss: 0.7586 - val_acc: 0.6505
Epoch 7/100
 - 3s - loss: 0.0668 - acc: 0.9811 - val_loss: 0.8230 - val_acc: 0.6626
Epoch 8/100
 - 3s - loss: 0.0790 - acc: 0.9817 - val_loss: 0.8809 - val_acc: 0.6505
Epoch 9/100
 - 3s - loss: 0.0676 - acc: 0.9805 - val_loss: 0.9055 - val_acc: 0.6505
Epoch 10/100
 - 3s - loss: 0.0485 - acc: 0.9856 - val_loss: 0.9346 - val_acc: 0.6562
Epoch 11/100
 - 3s - loss: 0.0452 - acc: 0.9874 - val_loss: 1.0438 - val_acc: 0.6457
Epoch 12/100
 - 3s - loss:

Epoch 97/100
 - 3s - loss: 0.0694 - acc: 0.9871 - val_loss: 1.9634 - val_acc: 0.6605
Epoch 98/100
 - 3s - loss: 0.0633 - acc: 0.9904 - val_loss: 2.0069 - val_acc: 0.6556
Epoch 99/100
 - 3s - loss: 0.0678 - acc: 0.9874 - val_loss: 2.1206 - val_acc: 0.6653
Epoch 100/100
 - 3s - loss: 0.0752 - acc: 0.9874 - val_loss: 1.9451 - val_acc: 0.6629
0.6222222222222222


In [16]:
def cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [17]:
print(cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 0.6885 - acc: 0.5603 - val_loss: 0.6527 - val_acc: 0.6004
Epoch 2/100
 - 2s - loss: 0.5970 - acc: 0.6742 - val_loss: 0.6212 - val_acc: 0.6349
Epoch 3/100
 - 2s - loss: 0.5029 - acc: 0.7577 - val_loss: 0.6086 - val_acc: 0.6487
Epoch 4/100
 - 2s - loss: 0.3838 - acc: 0.8326 - val_loss: 0.6521 - val_acc: 0.6490
Epoch 5/100
 - 2s - loss: 0.2591 - acc: 0.9167 - val_loss: 0.7122 - val_acc: 0.6502
Epoch 6/100
 - 2s - loss: 0.1729 - acc: 0.9501 - val_loss: 0.7241 - val_acc: 0.6713
Epoch 7/100
 - 2s - loss: 0.1201 - acc: 0.9745 - val_loss: 0.8014 - val_acc: 0.6538
Epoch 8/100
 - 2s - loss: 0.0890 - acc: 0.9817 - val_loss: 0.8829 - val_acc: 0.6553
Epoch 9/100
 - 2s - loss: 0.0839 - acc: 0.9802 - val_loss: 0.9059 - val_acc: 0.6538
Epoch 10/100
 - 2s - loss: 0.0908 - acc: 0.9790 - val_loss: 0.9449 - val_acc: 0.6562
Epoch 11/100
 - 2s - loss: 0.0822

Epoch 94/100
 - 2s - loss: 0.0524 - acc: 0.9793 - val_loss: 1.6517 - val_acc: 0.6505
Epoch 95/100
 - 2s - loss: 0.0272 - acc: 0.9886 - val_loss: 1.9583 - val_acc: 0.6460
Epoch 96/100
 - 2s - loss: 0.0213 - acc: 0.9904 - val_loss: 1.9926 - val_acc: 0.6469
Epoch 97/100
 - 2s - loss: 0.0183 - acc: 0.9904 - val_loss: 2.0230 - val_acc: 0.6502
Epoch 98/100
 - 2s - loss: 0.0169 - acc: 0.9913 - val_loss: 2.1494 - val_acc: 0.6436
Epoch 99/100
 - 2s - loss: 0.0177 - acc: 0.9919 - val_loss: 2.1838 - val_acc: 0.6472
Epoch 100/100
 - 2s - loss: 0.0195 - acc: 0.9907 - val_loss: 2.1973 - val_acc: 0.6451
0.6011482607227289


In [18]:
def cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [19]:
print(cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 3s - loss: 0.6882 - acc: 0.5705 - val_loss: 0.6273 - val_acc: 0.6406
Epoch 2/100
 - 2s - loss: 0.5348 - acc: 0.7322 - val_loss: 0.5684 - val_acc: 0.6605
Epoch 3/100
 - 2s - loss: 0.3162 - acc: 0.8921 - val_loss: 0.6531 - val_acc: 0.6517
Epoch 4/100
 - 2s - loss: 0.1492 - acc: 0.9594 - val_loss: 0.6542 - val_acc: 0.6728
Epoch 5/100
 - 2s - loss: 0.0826 - acc: 0.9817 - val_loss: 0.7468 - val_acc: 0.6605
Epoch 6/100
 - 2s - loss: 0.0723 - acc: 0.9844 - val_loss: 0.7619 - val_acc: 0.6617
Epoch 7/100
 - 2s - loss: 0.0607 - acc: 0.9862 - val_loss: 0.7645 - val_acc: 0.6677
Epoch 8/100
 - 2s - loss: 0.0602 - acc: 0.9853 - val_loss: 0.7752 - val_acc: 0.6650
Epoch 9/100
 - 2s - loss: 0.0554 - acc: 0.9844 - val_loss: 0.8269 - val_acc: 0.6532
Epoch 10/100
 - 2s - loss: 0.0502 - acc: 0.9880 - val_loss: 0.7961 - val_acc: 0.6668
Epoch 11/100
 - 2s - loss: 0.0410 - acc: 0.9880 - val_loss: 0.8235 - val_acc: 0.6602
Epoch 12/100
 - 2s - loss:

Epoch 97/100
 - 2s - loss: 0.0168 - acc: 0.9904 - val_loss: 1.7538 - val_acc: 0.6544
Epoch 98/100
 - 2s - loss: 0.0195 - acc: 0.9904 - val_loss: 1.6976 - val_acc: 0.6523
Epoch 99/100
 - 2s - loss: 0.0169 - acc: 0.9916 - val_loss: 1.8684 - val_acc: 0.6490
Epoch 100/100
 - 2s - loss: 0.0154 - acc: 0.9919 - val_loss: 1.8618 - val_acc: 0.6517
0.5799202609641173


In [20]:
def lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [21]:
print(lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 361s - loss: 0.6852 - acc: 0.5582 - val_loss: 0.6755 - val_acc: 0.5643
Epoch 2/10
 - 356s - loss: 0.6596 - acc: 0.6020 - val_loss: 0.6679 - val_acc: 0.6031
Epoch 3/10
 - 354s - loss: 0.6342 - acc: 0.6306 - val_loss: 0.6572 - val_acc: 0.6157
Epoch 4/10
 - 374s - loss: 0.6092 - acc: 0.6664 - val_loss: 0.6507 - val_acc: 0.6136
Epoch 5/10
 - 420s - loss: 0.5640 - acc: 0.7024 - val_loss: 0.6427 - val_acc: 0.6298
Epoch 6/10
 - 457s - loss: 0.5110 - acc: 0.7436 - val_loss: 0.6561 - val_acc: 0.6160
Epoch 7/10
 - 421s - loss: 0.4656 - acc: 0.7839 - val_loss: 0.6719 - val_acc: 0.6433
Epoch 8/10
 - 426s - loss: 0.3949 - acc: 0.8239 - val_loss: 0.7021 - val_acc: 0.6493
Epoch 9/10
 - 389s - loss: 0.3392 - acc: 0.8527 - val_loss: 0.7830 - val_acc: 0.6493
Epoch 10/10
 - 358s - loss: 0.2809 - acc: 0.8822 - val_loss: 0.7911 - val_acc: 0.6484
0.5987654320987655


In [22]:
def lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [23]:
print(lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 405s - loss: 0.6893 - acc: 0.5488 - val_loss: 0.6771 - val_acc: 0.5676
Epoch 2/10
 - 392s - loss: 0.6532 - acc: 0.6207 - val_loss: 0.6603 - val_acc: 0.5968
Epoch 3/10
 - 388s - loss: 0.5947 - acc: 0.6850 - val_loss: 0.6435 - val_acc: 0.6130
Epoch 4/10
 - 390s - loss: 0.4946 - acc: 0.7686 - val_loss: 0.6822 - val_acc: 0.6397
Epoch 5/10
 - 431s - loss: 0.3884 - acc: 0.8266 - val_loss: 0.7693 - val_acc: 0.6433
Epoch 6/10
 - 391s - loss: 0.2799 - acc: 0.8795 - val_loss: 0.8546 - val_acc: 0.6529
Epoch 7/10
 - 365s - loss: 0.1876 - acc: 0.9261 - val_loss: 1.0346 - val_acc: 0.6587
Epoch 8/10
 - 362s - loss: 0.1221 - acc: 0.9567 - val_loss: 1.3248 - val_acc: 0.6566
Epoch 9/10
 - 364s - loss: 0.0967 - acc: 0.9621 - val_loss: 1.4780 - val_acc: 0.6478
Epoch 10/10
 - 362s - loss: 0.0786 - acc: 0.9732 - val_loss: 1.3214 - val_acc: 0.6505
0.5753924790069368
